In [2]:
from pathlib import Path
import rasterio
import numpy as np

import matplotlib.pyplot as plt
dataset = Path("/eodc/private/tuwgeo/users/mabdelaa/data/watmap_test_cases_final")
scenes = [dir for dir in dataset.iterdir() if dir.is_dir()]

In [ ]:
import numpy as np
import rasterio

# Thresholds
correct_thresh = 0.5
certainty_thresh = 0.7

# Store totals across all scenes (percentages)
s1_totals = np.zeros(5, dtype=np.float64)  # classes 0-4
s2_totals = np.zeros(5, dtype=np.float64)
scene_counter = 0
s1_scene_per_avg = np.zeros(5, dtype=np.float64)
s2_scene_per_avg = np.zeros(5, dtype=np.float64)
for scene in scenes:
    print(f"\nProcessing scene: {scene}")
    scene_counter += 1

    # Collect files
    mask_files = sorted((scene / "mask").glob("*.tif"))
    s1_mu_files = sorted((scene / "s1_mu").glob("*.tif"))
    s1_entropy_files = sorted((scene / "s1_entropy").glob("*.tif"))
    s2_mu_files = sorted((scene / "s2_mu").glob("*.tif"))
    s2_entropy_files = sorted((scene / "s2_entropy").glob("*.tif"))

    # Iterate over files in the scene
    for mask_file, s1_mu_file, s1_ent_file, s2_mu_file, s2_ent_file in zip(
        mask_files, s1_mu_files, s1_entropy_files, s2_mu_files, s2_entropy_files
    ):
        # --- Load rasters ---
        with rasterio.open(mask_file) as src:
            mask = src.read(1)
            profile = src.profile
        with rasterio.open(s1_mu_file) as src:
            s1_mu = src.read(1)
        with rasterio.open(s1_ent_file) as src:
            s1_entropy = src.read(1)
        with rasterio.open(s2_mu_file) as src:
            s2_mu = src.read(1)
        with rasterio.open(s2_ent_file) as src:
            s2_entropy = src.read(1)

        profile.update(dtype=rasterio.uint8, count=1, compress="lzw")

        # --- Sentinel-1 classification ---
        s1_final = np.zeros_like(mask, dtype=np.uint8)
        s1_correct = ((mask == 1) & (s1_mu >= correct_thresh)) | \
                     ((mask == 0) & (s1_mu <  correct_thresh))
        s1_incorrect = ~s1_correct

        s1_certain   = (s1_entropy >= certainty_thresh)
        s1_uncertain = (s1_entropy <  certainty_thresh)

        s1_final[s1_correct & s1_certain]     = 1
        s1_final[s1_correct & s1_uncertain]   = 2
        s1_final[s1_incorrect & s1_certain]   = 3
        s1_final[s1_incorrect & s1_uncertain] = 4

        # --- Sentinel-2 classification ---
        s2_final = np.zeros_like(mask, dtype=np.uint8)
        s2_correct = ((mask == 1) & (s2_mu >= correct_thresh)) | \
                     ((mask == 0) & (s2_mu <  correct_thresh))
        s2_incorrect = ~s2_correct

        s2_certain   = (s2_entropy >= certainty_thresh)
        s2_uncertain = (s2_entropy <  certainty_thresh)

        s2_final[s2_correct & s2_certain]     = 1
        s2_final[s2_correct & s2_uncertain]   = 2
        s2_final[s2_incorrect & s2_certain]   = 3
        s2_final[s2_incorrect & s2_uncertain] = 4

        # # --- Save outputs ---
        # s1_save_dir = scene / "s1_uncer_quan"
        # s1_save_dir.mkdir(exist_ok=True)
        # s1_save_path = s1_save_dir / mask_file.name.replace("mask", "s1_uncer_quan")
        # with rasterio.open(s1_save_path, "w", **profile) as dst:
        #     dst.write(s1_final, 1)

        # s2_save_dir = scene / "s2_uncer_quan"
        # s2_save_dir.mkdir(exist_ok=True)
        # s2_save_path = s2_save_dir / mask_file.name.replace("mask", "s2_uncer_quan")
        # with rasterio.open(s2_save_path, "w", **profile) as dst:
        #     dst.write(s2_final, 1)

        # print(f"Saved S1: {s1_save_path}")
        # print(f"Saved S2: {s2_save_path}")

        # --- Stats per scene ---
        s1_classes, s1_counts = np.unique(s1_final, return_counts=True)
        s2_classes, s2_counts = np.unique(s2_final, return_counts=True)

        s1_total = np.sum(s1_counts)
        s2_total = np.sum(s2_counts)

        # Compute percentages for this scene
        s1_percentages = np.zeros(5, dtype=np.float64)
        s2_percentages = np.zeros(5, dtype=np.float64)

        for cls, count in zip(s1_classes, s1_counts):
            perc = (count / s1_total) * 100
            s1_percentages[cls] = perc
            print(f"S1 - Class {cls}: {perc:.2f}%")
        print("\n")

        for cls, count in zip(s2_classes, s2_counts):
            perc = (count / s2_total) * 100
            s2_percentages[cls] = perc
            print(f"S2 - Class {cls}: {perc:.2f}%")
        print("\n")
        print("\n---\n")

        # --- Accumulate percentages (not counts) ---
        s1_totals += s1_percentages
        s2_totals += s2_percentages
    
    s1_scene_per_avg += s1_totals / 16
    s2_scene_per_avg += s2_totals / 16
    
    s1_totals = np.zeros(5, dtype=np.float64)  # reset for next scene
    s2_totals = np.zeros(5, dtype=np.float64)
    
    print(f"\n=== Average percentages for scene {scene.name} ===")
    for cls in range(5):
        print(f"S1 - Class {cls}: {s1_scene_per_avg[cls]:.2f}%")
        print(f"S2 - Class {cls}: {s2_scene_per_avg[cls]:.2f}%")




Processing scene: /eodc/private/tuwgeo/users/mabdelaa/data/watmap_test_cases_final/80
S1 - Class 0: 98.78%
S1 - Class 1: 0.66%
S1 - Class 2: 0.45%
S1 - Class 3: 0.03%
S1 - Class 4: 0.07%


S2 - Class 1: 98.54%
S2 - Class 2: 1.16%
S2 - Class 3: 0.06%
S2 - Class 4: 0.25%



---

S1 - Class 0: 99.69%
S1 - Class 1: 0.07%
S1 - Class 2: 0.15%
S1 - Class 3: 0.02%
S1 - Class 4: 0.06%


S2 - Class 1: 99.68%
S2 - Class 2: 0.29%
S2 - Class 3: 0.00%
S2 - Class 4: 0.03%



---

S1 - Class 0: 94.22%
S1 - Class 1: 4.98%
S1 - Class 2: 0.54%
S1 - Class 3: 0.06%
S1 - Class 4: 0.20%


S2 - Class 1: 98.23%
S2 - Class 2: 1.32%
S2 - Class 3: 0.01%
S2 - Class 4: 0.45%



---

S1 - Class 0: 98.23%
S1 - Class 1: 1.12%
S1 - Class 2: 0.24%
S1 - Class 3: 0.11%
S1 - Class 4: 0.30%


S2 - Class 1: 99.02%
S2 - Class 2: 0.76%
S2 - Class 3: 0.01%
S2 - Class 4: 0.21%



---

S1 - Class 0: 99.91%
S1 - Class 1: 0.01%
S1 - Class 2: 0.08%
S1 - Class 3: 0.00%
S1 - Class 4: 0.01%


S2 - Class 1: 99.62%
S2 - Class 2: 0.35%
S

KeyboardInterrupt: 

In [72]:
# --- Average percentages across all scenes ---
s1_avg = s1_scene_per_avg / scene_counter
s2_avg = s2_scene_per_avg / scene_counter

print("\n=== Average percentages across all scenes ===")
for cls in range(5):
    print(f"S1 - Class {cls}: {s1_avg[cls]:.2f}%")
    print(f"S2 - Class {cls}: {s2_avg[cls]:.2f}%")


=== Average percentages across all scenes ===
S1 - Class 0: 0.00%
S2 - Class 0: 0.00%
S1 - Class 1: 94.10%
S2 - Class 1: 88.51%
S1 - Class 2: 2.47%
S2 - Class 2: 9.49%
S1 - Class 3: 2.45%
S2 - Class 3: 0.11%
S1 - Class 4: 0.98%
S2 - Class 4: 1.89%
